In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
public_key = 'f20f7f3c-f092-4814-96db-0999e0d71b08'
private_key = 'W/aIoaGsZoYJouTP3ppnjYuHNyJmxy9s'

In [3]:
import sys
sys.path.append('../')

In [4]:
import requests
import threading

import src
from src.websocket_api import handlers
from src.websocket_api import listeners
import subprocess
import logging
logging.basicConfig(level=logging.INFO)

In [5]:
items = list()
for symbol in ['try']:
    response = requests.get(f'https://api.btcturk.com/api/v2/ticker/currency?symbol={symbol}')
    for item in response.json()['data']:
        items.append(item['pair'])

In [6]:
results = list()

In [7]:
def on_message(ws, message):
    result = handlers.handle_channel_message(message)
    global results
    results += [result]
    
def on_error(*args):
    print(args)
    print()

def on_close(*args):
    print(args)

In [8]:
threads = list()

In [9]:
for item in items:
    _thread = threading.Thread(
        target=listeners.get_listener,
        kwargs={
            'on_message': on_message,
            'on_close': on_close,
            'on_error': on_error,
            'event': item,
            'channel': 'trade',
            'time_delay': 1
        }
    )
    threads += [_thread]

In [10]:
def thread_control():
    
    while True:
        import time
        global threads
        global results

        counter = 0

        for thread in threads:
            if thread.is_alive():
                continue
            else:
                counter += 1
                thread.run()

        print(f'Started threads {counter}')
        print(f'Total records {len(results)}')

        time.sleep(5)

In [11]:
control = threading.Thread(
    target=thread_control,
)

In [12]:
control.start()

In [18]:
len(results)

721

In [58]:
import pickle

In [ ]:
with open('orderbook', 'wb') as f:
    pickle.dump(results, f)